In [1]:
%load_ext autoreload
%autoreload 2

# Initialize

In [2]:
import dspy

In [3]:
# Clear DSPy's global disk cache
if hasattr(dspy, 'cache') and hasattr(dspy.cache, 'disk_cache'):
    dspy.cache.disk_cache.clear()
    print("clear")

clear


In [4]:
from package.base import DriverLM, ModelResponse, Usage
import httpx

ollama_client = httpx.Client(timeout=600.0)

def ollama_request_fn(prompt: str | None = None, messages: list[dict] | None = None, temperature: float = 0.0, max_tokens: int = 256) -> dict:
    if messages is None:
        messages = [{"role": "user", "content": prompt}]
    
    response = ollama_client.post(
        'http://localhost:11434/api/chat',
        json={
            "model": "llama3.2-vision:11b",
            "messages": messages,
            "stream": False,
            "options": {"temperature": temperature}
        }
    )
    response.raise_for_status()
    return response.json()  # Return full Ollama response

def ollama_output_fn(response: dict) -> ModelResponse:
    content = response.get("message", {}).get("content", "")
    model = response.get("model", "custom")
    
    usage = Usage(
        prompt_tokens=response.get("prompt_eval_count", 0),
        completion_tokens=response.get("eval_count", 0),
        total_tokens=response.get("prompt_eval_count", 0) + response.get("eval_count", 0)
    )
    
    return ModelResponse.from_text(text=content.strip(), usage=usage, model=model)


custom_lm = DriverLM(
    request_fn=ollama_request_fn,
    output_fn=ollama_output_fn,
    cache=True
)

custom_lm.clear_cache()  # Clear old cache entries

# Fewshots tuning with BootstrapFewShot

In [5]:
from dspy.teleprompt import BootstrapFewShot

# Configure your DriverLM
dspy.configure(lm=custom_lm)

# Define signature
class Emotion(dspy.Signature):
    """Classify the emotion of a sentence."""
    sentence = dspy.InputField()
    emotion = dspy.OutputField(desc="happy, sad, or neutral")

# Create training data
trainset = [
    dspy.Example(sentence="I love this!", emotion="happy").with_inputs("sentence"),
    dspy.Example(sentence="This is terrible.", emotion="sad").with_inputs("sentence"),
    dspy.Example(sentence="It's okay.", emotion="neutral").with_inputs("sentence"),
]

# Create module
class EmotionClassifier(dspy.Module):
    def __init__(self):
        self.program = dspy.ChainOfThought(Emotion)
    
    def forward(self, sentence):
        return self.program(sentence=sentence)

# Optimize
optimizer = BootstrapFewShot(max_bootstrapped_demos=2, max_labeled_demos=2)
optimized = optimizer.compile(EmotionClassifier(), trainset=trainset)

# Test
result = optimized(sentence="I'm so excited!")
print(f"Emotion: {result.emotion}")
print(f"Reasoning: {result.reasoning}")
optimized(sentence="I love pizza")

 67%|██████▋   | 2/3 [00:01<00:00,  1.29it/s]


Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Emotion: happy
Reasoning: The use of the word "excited" and the exclamation mark at the end of the sentence indicate a strong positive emotion.


Prediction(
    reasoning='The sentence expresses a positive emotion, indicating that the speaker enjoys or has a strong affection for pizza.',
    emotion='happy'
)

In [6]:
optimized(sentence="I love pizza")

Prediction(
    reasoning='The sentence expresses a positive emotion, indicating that the speaker enjoys or has a strong affection for pizza.',
    emotion='happy'
)

In [7]:
from pathlib import Path

artifact_dir = Path("./artifacts")
artifact_dir.mkdir(parents=True, exist_ok=True)

# 1. Inspect the optimized prompts
print("=== Optimized Module ===")
for name, predictor in optimized.named_predictors():
    print(f"\n{name}:")
    print(f"Signature: {predictor.signature}")
    if hasattr(predictor, 'demos'):
        print(f"Demos: {len(predictor.demos)} examples")
        for i, demo in enumerate(predictor.demos):
            print(f"  Demo {i+1}: {demo}")

# 2. Save the optimized module
filename= artifact_dir / "fewshots_emotion_custom_classifier.json"
optimized.save(filename)
print(f"\nSaved to {filename}")

# 3. Load it later
loaded = EmotionClassifier()
loaded.load(filename)

# 4. Simple inspection
print("\n=== Quick Check ===")
print(f"Module type: {type(optimized)}")
print(f"Has program: {hasattr(optimized, 'program')}")

=== Optimized Module ===

program.predict:
Signature: StringSignature(sentence -> reasoning, emotion
    instructions='Classify the emotion of a sentence.'
    sentence = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Sentence:', 'desc': '${sentence}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    emotion = Field(annotation=str required=True json_schema_extra={'desc': 'happy, sad, or neutral', '__dspy_field_type': 'output', 'prefix': 'Emotion:'})
)
Demos: 2 examples
  Demo 1: Example({'augmented': True, 'sentence': 'I love this!', 'reasoning': 'The exclamation mark at the end of the sentence indicates a strong positive emotion, and the word "love" is typically associated with affection or fondness.', 'emotion': 'happy'}) (input_keys=None)
  Demo 2: Example({'augmented': True, 'sentence': 'This i

In [8]:
loaded(sentence="I love pizza")

Prediction(
    reasoning='The sentence expresses a positive emotion, indicating that the speaker enjoys or has a strong affection for pizza.',
    emotion='happy'
)